In [1]:
import datetime
import os
import pathlib
import warnings

import load_ribasim  # noqa: F401
import pandas as pd
import ribasim
from ribasim import Node
from ribasim.nodes import pump, level_boundary, tabulated_rating_curve
from shapely import Point

import peilbeheerst_model.ribasim_parametrization as ribasim_param
from peilbeheerst_model.add_storage_basins import AddStorageBasins
from peilbeheerst_model.controle_output import Control
from peilbeheerst_model.ribasim_feedback_processor import RibasimFeedbackProcessor

%reload_ext autoreload
%autoreload 2
warnings.filterwarnings("ignore")

## Define variables and model

#### Set Config

In [2]:
waterschap = "Zuiderzeeland"
work_dir = pathlib.Path(f"../../../../../Ribasim_updated_models/{waterschap}/modellen/{waterschap}_parametrized")
ribasim_gpkg = work_dir.joinpath("database.gpkg")
path_ribasim_toml = work_dir.joinpath("ribasim.toml")
output_dir = work_dir.joinpath("results")

# Basin area percentage
regular_percentage = 10
boezem_percentage = 90
unknown_streefpeil = (
    0.00012345  # we need a streefpeil to create the profiles, Q(h)-relations, and af- and aanslag peil for pumps
)

# Forcing settings
start_time = "2024-01-01"
timestep_size = "d"
timesteps = 2
delta_crest_level = 0.1  # delta waterlevel of boezem compared to streefpeil till no water can flow through an outlet

default_level = -0.40  # default LevelBoundary level, similar to surrounding IJsselmeer, Markermeer and Randmeren

## Process the feedback form

In [3]:
name = "Ron Bruijns (HKV)"
versie = "2024_12_0"

feedback_excel = pathlib.Path(f"../../../../../Ribasim_feedback/V1_formulieren/feedback_formulier_{waterschap}.xlsx")
feedback_excel_processed = (
    f"../../../../..//Ribasim_feedback/V1_formulieren_verwerkt/feedback_formulier_{waterschap}_JA_processed.xlsx"
)

ribasim_toml = f"../../../../../Ribasim_base_models/{waterschap}_boezemmodel_{versie}/ribasim.toml"
output_folder = work_dir  # f"../../../../../Ribasim_updated_models/{waterschap}"

processor = RibasimFeedbackProcessor(
    name, waterschap, versie, feedback_excel, ribasim_toml, output_folder, feedback_excel_processed, use_validation=True
)
processor.run()

Swapped edge direction between Node A: 16 and Node B: 816
Swapped edge direction between Node A: 816 and Node B: 835
Swapped edge direction between Node A: 817 and Node B: 316
Swapped edge direction between Node A: 316 and Node B: 23
Swapped edge direction between Node A: 818 and Node B: 322
Swapped edge direction between Node A: 322 and Node B: 16
Swapped edge direction between Node A: 823 and Node B: 489
Swapped edge direction between Node A: 489 and Node B: 133
Swapped edge direction between Node A: 827 and Node B: 658
Swapped edge direction between Node A: 658 and Node B: 119
Swapped edge direction between Node A: 821 and Node B: 466
Swapped edge direction between Node A: 466 and Node B: 119
Swapped edge direction between Node A: 819 and Node B: 403
Swapped edge direction between Node A: 403 and Node B: 88
Swapped edge direction between Node A: 817 and Node B: 316
Swapped edge direction between Node A: 316 and Node B: 23
Swapped edge direction between Node A: 826 and Node B: 514
Sw

#### Load model

In [4]:
# Load Ribasim model
with warnings.catch_warnings():
    warnings.simplefilter(action="ignore", category=FutureWarning)
    ribasim_model = ribasim.Model(filepath=path_ribasim_toml)

# Parameterization

## Nodes

### Basin (characteristics)

In [5]:
ribasim_param.validate_basin_area(ribasim_model)

All basins are larger than 100 m²


In [6]:
# remove the basins of above in the feedback form

## Model specific tweaks

In [7]:
#1 Add gemaal at blocq van kuffeler
new_node_id = max(ribasim_model.edge.df.from_node_id.max(), ribasim_model.edge.df.to_node_id.max()) + 1
level_boundary_node = ribasim_model.level_boundary.add(Node(new_node_id, Point(143912, 492256)), [level_boundary.Static(level=[default_level])])

pump_node = ribasim_model.pump.add(Node(new_node_id+1, Point(143959, 492198)),[pump.Static(flow_rate=[0.1])])

ribasim_model.edge.add(ribasim_model.basin[31], pump_node)
ribasim_model.edge.add(pump_node, level_boundary_node)

#1 Add gemaal Wortman
new_node_id = max(ribasim_model.edge.df.from_node_id.max(), ribasim_model.edge.df.to_node_id.max()) + 1
level_boundary_node = ribasim_model.level_boundary.add(Node(new_node_id, Point(157201, 501796)), [level_boundary.Static(level=[default_level])])

pump_node = ribasim_model.pump.add(Node(new_node_id+1, Point(157251, 501708)),[pump.Static(flow_rate=[0.1])])

ribasim_model.edge.add(ribasim_model.basin[31], pump_node)
ribasim_model.edge.add(pump_node, level_boundary_node)

#Inlaat (hevel) toevoegen
new_node_id = max(ribasim_model.edge.df.from_node_id.max(), ribasim_model.edge.df.to_node_id.max()) + 1
level_boundary_node = ribasim_model.level_boundary.add(Node(new_node_id, Point(193502,526518)), [level_boundary.Static(level=[default_level])])

tabulated_rating_curve_node = ribasim_model.tabulated_rating_curve.add(Node(new_node_id+1, Point(193491,526526)),
                                                                      [tabulated_rating_curve.Static(
                                                                        level=[0.0, 0.1234],
                                                                        flow_rate=[0.0, 0.1234])])
ribasim_model.edge.add(level_boundary_node, tabulated_rating_curve_node)
ribasim_model.edge.add(tabulated_rating_curve_node, ribasim_model.basin[98])


#Inlaat (hevel) toevoegen
new_node_id = max(ribasim_model.edge.df.from_node_id.max(), ribasim_model.edge.df.to_node_id.max()) + 1
level_boundary_node = ribasim_model.level_boundary.add(Node(new_node_id, Point(185725,533120)), [level_boundary.Static(level=[default_level])])

tabulated_rating_curve_node = ribasim_model.tabulated_rating_curve.add(Node(new_node_id+1, Point(185725,533098)),
                                                                      [tabulated_rating_curve.Static(
                                                                        level=[0.0, 0.1234],
                                                                        flow_rate=[0.0, 0.1234])])
ribasim_model.edge.add(level_boundary_node, tabulated_rating_curve_node)
ribasim_model.edge.add(tabulated_rating_curve_node, ribasim_model.basin[116])

In [8]:
ribasim_model.level_boundary.node.df.meta_node_id = ribasim_model.level_boundary.node.df.index
ribasim_model.tabulated_rating_curve.node.df.meta_node_id = ribasim_model.tabulated_rating_curve.node.df.index
ribasim_model.pump.node.df.meta_node_id = ribasim_model.pump.node.df.index



In [9]:
# change unknown streefpeilen to a default streefpeil
ribasim_model.basin.area.df.loc[
    ribasim_model.basin.area.df["meta_streefpeil"] == "Onbekend streefpeil", "meta_streefpeil"
] = str(unknown_streefpeil)
ribasim_model.basin.area.df.loc[ribasim_model.basin.area.df["meta_streefpeil"] == -9.999, "meta_streefpeil"] = str(
    unknown_streefpeil
)

In [10]:
assert not pd.isnull(ribasim_model.basin.area.df.meta_streefpeil).any()

## Implement standard profile and a storage basin

In [11]:
# Insert standard profiles to each basin. These are [depth_profiles] meter deep, defined from the streefpeil
ribasim_param.insert_standard_profile(
    ribasim_model,
    unknown_streefpeil=unknown_streefpeil,
    regular_percentage=regular_percentage,
    boezem_percentage=boezem_percentage,
    depth_profile=2,
)

In [12]:
add_storage_basins = AddStorageBasins(
    ribasim_model=ribasim_model, exclude_hoofdwater=True, additional_basins_to_exclude=[]
)

add_storage_basins.create_bergende_basins()


### Basin (forcing)

In [13]:
# Set static forcing
forcing_dict = {
    "precipitation": ribasim_param.convert_mm_day_to_m_sec(0),
    "potential_evaporation": ribasim_param.convert_mm_day_to_m_sec(10),
    "drainage": ribasim_param.convert_mm_day_to_m_sec(0),
    "infiltration": ribasim_param.convert_mm_day_to_m_sec(0),
    # 'urban_runoff':          ribasim_param.convert_mm_day_to_m_sec(0),
}

ribasim_param.set_static_forcing(timesteps, timestep_size, start_time, forcing_dict, ribasim_model)

### Pumps

In [14]:
# Set pump capacity for each pump
ribasim_model.pump.static.df["flow_rate"] = 0.16667  # 10 kuub per minuut

### Convert all boundary nodes to LevelBoundaries

In [15]:
ribasim_param.Terminals_to_LevelBoundaries(ribasim_model=ribasim_model, default_level=default_level)  # clean
ribasim_param.FlowBoundaries_to_LevelBoundaries(ribasim_model=ribasim_model, default_level=default_level)

### Add Outlet

In [16]:
ribasim_param.add_outlets(ribasim_model, delta_crest_level=0.10)

## Add control, based on the meta_categorie

In [17]:
ribasim_param.identify_node_meta_categorie(ribasim_model)

In [18]:
ribasim_param.find_upstream_downstream_target_levels(ribasim_model, node="outlet")
ribasim_param.find_upstream_downstream_target_levels(ribasim_model, node="pump")


In [19]:
# ribasim_param.add_discrete_control(ribasim_model, waterschap, default_level)

In [20]:
ribasim_param.determine_min_upstream_max_downstream_levels(ribasim_model, waterschap)

Warning! Some pumps do not have a flow rate yet. Dummy value of 0.1234 m3/s has been taken.


### Manning Resistance

In [21]:
# there is a MR without geometry and without edges for some reason
ribasim_model.manning_resistance.node.df = ribasim_model.manning_resistance.node.df.dropna(subset="geometry")

In [22]:
#lower the difference in waterlevel for each manning node
ribasim_model.manning_resistance.static.df.length = 10
ribasim_model.manning_resistance.static.df.manning_n = 0.01

## Last formating of the tables

In [23]:
# only retain node_id's which are present in the .node table
ribasim_param.clean_tables(ribasim_model, waterschap)

# Set numerical settings

In [24]:
# Write model output
ribasim_model.use_validation = True
ribasim_model.starttime = datetime.datetime(2024, 1, 1)
ribasim_model.endtime = datetime.datetime(2025, 1, 1)
ribasim_model.solver.saveat = 3600
ribasim_param.write_ribasim_model_Zdrive(ribasim_model, path_ribasim_toml)

## Run Model

In [25]:
ribasim_param.tqdm_subprocess(["ribasim", path_ribasim_toml], 
                              print_other=False, 
                              suffix="init")

Simulating init:   0%|          | 0/100 [00:00<?, ?it/s]

In [26]:
controle_output = Control(work_dir=work_dir)
indicators = controle_output.run_all()

PermissionError: [Errno 1] Operation not permitted: '../../../../../Ribasim_updated_models/Zuiderzeeland/modellen/Zuiderzeeland_parametrized/results/output_controle.qlr'

# Write model

In [ ]:
# control_dict = Control(work_dir = work_dir).run_all()
ribasim_param.write_ribasim_model_GoodCloud(
    ribasim_model=ribasim_model,
    path_ribasim_toml=path_ribasim_toml,
    waterschap=waterschap,
    modeltype="boezemmodel",
    include_results=True,
)